In [12]:
import pandas as pd 
import pickle
import numpy as np
user_data_file= "Data_Sci_Challenge_Datasets/Data_science_challenge_files/User_scores.csv"

In [5]:
#define maps
map_intrests = {
    "no high point available": 0,
    "realistic": 1,
    "investigative": 2,
    "artistic": 3,
    "social":4,
    "enterprising":5,
    "conventional":6,
    "no high point available":0,
    "achievement": 1,
    "working Conditions": 2,
    "recognition": 3,
    "relationships": 4,
    "support": 5,
    "independence": 6

}
lst_labels_tops=[ 
       'third interest high-point',
       'second interest high-point', 
       'first interest high-point',
       'third work value high-point',
       'second work value high-point',
       'first work value high-point',
        ]

In [8]:
#load user data 
df_user = pd.read_csv(user_data_file)

#extract extra features for inference, fix data names, prepare input
df_tops =df_user.sort_values('score').groupby(['user_id','element_group'], sort=True).tail(3)
df_tops["tops"] = lst_labels_tops
df_tops["tops_values"] = df_tops["element_name"].map(lambda x : map_intrests[x] )
df_tops = df_tops.pivot(index="user_id", columns="tops",values="tops_values")
df_user= df_user.pivot(index="user_id", columns="element_name",values="score")
df_user_norm = df_user.join(df_tops,on='user_id',how="inner")
df_user_norm = df_user_norm.reindex(sorted(df_user_norm.columns), axis=1)

In [13]:
with open('model.pickle', 'rb') as file:
    fitted = pickle.load(file)

with open('label_encoder.pickle', 'rb') as file:
    label_encoder = pickle.load(file)



In [14]:
res= fitted.predict_proba(df_user_norm)

In [15]:
top_10= np.argpartition(res[0], -10)[-10:]
top_10_jobs = label_encoder.inverse_transform(top_10)
print(top_10_jobs)

['Critical Care Nurses' 'Medical and Health Services Managers'
 'Family Medicine Physicians' 'Psychiatrists' 'Pediatricians, General'
 'Occupational Therapists'
 'Education Administrators, Kindergarten through Secondary'
 'Chief Executives' 'Judges, Magistrate Judges, and Magistrates'
 'Nurse Practitioners']
